In [ ]:
class MyIterator(object):
 def __init__(self, xs):
    self.xs = xs
 def __iter__(self):
    return self
 def __next__(self):
    if self.xs:
      return self.xs.pop(0)
    else:
      print('end')
      raise StopIteration



In [ ]:
#iterator

In [ ]:
n=[1,2,3,4]
x = MyIterator(n)

for i in x :
  print(i)

1
2
3
4
end


In [ ]:
n=[1,2,3,4]
x = MyIterator(n)

print (next(x))

1


In [ ]:

print (next(x))

2


In [ ]:
#generator

In [ ]:
def gnrtr(n):
  while n :
    n-=1
    yield n

gnrtr(3)

<generator object gnrtr at 0x7b508c2a95b0>

In [ ]:
g = gnrtr(8)
g

<generator object gnrtr at 0x7b508c2a9070>

In [ ]:
next(g)

7

In [ ]:
next(g)

6

In [ ]:
next(g)

5

In [ ]:
!curl -sO https://www.gutenberg.org/cache/epub/100/pg100.txt.utf8

In [ ]:
!ls

pg100.txt.utf8	sample_data


In [ ]:
!time(grep -io macbeth pg100.txt.utf8 | wc -l)

288

real	0m0.013s
user	0m0.012s
sys	0m0.001s


In [ ]:
s ="""

def coroutine(fn):
  def wrapper(*args, **kwargs):
    c = fn(*args,**kwargs)
    next(c)
    return c
  return wrapper

def cat(f,child):
  #reads file line by line
  line_processor = lambda l : l.lower()
  for line in f :
    child.send(line_processor(line))

@coroutine
def grep(substring,child):
  #counts occurrences of substring in each line
  substring = substring.lower()
  while True:
    text = (yield)
    child.send(text.count(substring))

@coroutine
def count(substring):
  #add occurences to calculate total count
 n = 0
 try:
   while True:
      n += (yield)
 except GeneratorExit:
    print(substring, n)

if __name__ == '__main__':
 import argparse
 parser = argparse.ArgumentParser()
 parser.add_argument('pattern', type=str)
 parser.add_argument('infile', type=argparse.FileType('r'))
 args = parser.parse_args()
 cat(args.infile,  grep(args.pattern,  count(args.pattern)))





"""

text_file = open("grep.py", "w")
n = text_file.write(s)
text_file.close()

In [ ]:
!ls

grep.py  pg100.txt.utf8  sample_data


In [ ]:
!time(python grep.py macbeth pg100.txt.utf8)

macbeth 288

real	0m0.128s
user	0m0.118s
sys	0m0.004s


In [ ]:
!time(grep -io macbeth pg100.txt.utf8 | wc -l)

288

real	0m0.013s
user	0m0.010s
sys	0m0.002s


In [ ]:
s ="""

def coroutine(fn):
  def wrapper(*args, **kwargs):
    c = fn(*args,**kwargs)
    next(c)
    return c
  return wrapper

def cat(f,child):
  #reads file line by line
  line_processor = lambda l : l.lower()
  for line in f :
    child.send(line_processor(line))

@coroutine
def grep(substring,child):
  #counts occurrences of substring in each line
  substring = substring.lower()
  while True:
    text = (yield)
    child.send(text.count(substring))

@coroutine
def count(substring):
  #add occurences to calculate total count
 n = 0
 try:
   while True:
      n += (yield)
 except GeneratorExit:
    print(substring, n)


@coroutine
def fanout(children):
  #add occurences to calculate total count
  while True:
    data = (yield)
    for child in children:
      child.send(data)

if __name__ == '__main__':
 import argparse
 parser = argparse.ArgumentParser()
 parser.add_argument('pattern', type=str,nargs='+',)
 parser.add_argument('infile', type=argparse.FileType('r'))
 args = parser.parse_args()
 cat(args.infile,fanout([grep(p,  count(p))for p in args.pattern])  )





"""

text_file = open("fangrep.py", "w")
n = text_file.write(s)
text_file.close()

In [ ]:
!time(python fangrep.py doth macbeth love and water smoke sailor pg100.txt.utf8)

smoke 30
water 240
and 34909
love 3180
macbeth 288
doth 1083
sailor 45

real	0m0.439s
user	0m0.426s
sys	0m0.010s


In [ ]:
#parallelism

In [ ]:
#multiple threads

In [ ]:
## download yahoo finance data for multiple tickers
## using threads

In [ ]:
s ="""
from threading import Thread
from queue import Queue
import urllib.request
import pandas as pd
from io import StringIO as sio
import csv

URL ='https://query1.finance.yahoo.com/v7/finance/download/{}?period1=1660768315&period2=1692254623&interval=1d&events=history&includeAdjustedClose=true'

def get_rate(pair, outq, url_tmplt=URL):
  #download data from yahoo given a tickr
  #put data in que when recieved
 with urllib.request.urlopen(url_tmplt.format(pair)) as res:

  body = str(res.read())
  outq.put((pair, body.strip()))


if __name__ == '__main__':
 import argparse
 parser = argparse.ArgumentParser()
 parser.add_argument('pairs', type=str, nargs='+')
 args = parser.parse_args()

 #hold output data in this que
 outputq = Queue()

for pair in args.pairs:
  t= Thread(target=get_rate,kwargs={'pair':pair,
                                    'outq':outputq})

  #python wont wait for thread to quit
  #notice order of input vs order of putput(sequence)
  t.daemon=True
  t.start()

for _ in args.pairs:
  pair,rate=outputq.get()
  print(pair,rate)
  outputq.task_done()

outputq.join()
"""
text_file = open("currency.py", "w")
n = text_file.write(s)
text_file.close()

In [ ]:
!time python currency.py COIN TATAMOTORS.ns TSLA

TSLA b'Date,Open,High,Low,Close,Adj Close,Volume\n2022-08-17,303.396667,309.656677,300.033325,303.996674,303.996674,68766000\n2022-08-18,306.000000,306.500000,301.853333,302.869995,302.869995,47500500\n2022-08-19,299.000000,300.359985,292.500000,296.666656,296.666656,61395300\n2022-08-22,291.913330,292.399994,286.296661,289.913330,289.913330,55843200\n2022-08-23,291.453339,298.826660,287.923340,296.453339,296.453339,63984900\n2022-08-24,297.563324,303.646667,296.500000,297.096680,297.096680,57259800\n2022-08-25,302.359985,302.959991,291.600006,296.070007,296.070007,53230000\n2022-08-26,297.429993,302.000000,287.470001,288.089996,288.089996,57163900\n2022-08-29,282.829987,287.739990,280.700012,284.820007,284.820007,41864700\n2022-08-30,287.869995,288.480011,272.649994,277.700012,277.700012,50541800\n2022-08-31,280.619995,281.250000,271.809998,275.609985,275.609985,52107300\n2022-09-01,272.579987,277.579987,266.149994,277.160004,277.160004,54287000\n2022-09-02,281.070007,282.350006,269.0

In [ ]:
#MULTI PROCESSES

In [ ]:

s ="""
import concurrent.futures as cf

def fib(n):
  if n<=2:
    return 1
  if n==0:
    return 0
  return fib(n-1)+fib(n-2)


if __name__ == '__main__':
 import argparse
 parser = argparse.ArgumentParser()
 parser.add_argument('-n', type=int, default=1)
 parser.add_argument('number', type=int, nargs='?', default=34)
 args = parser.parse_args()
 assert args.n >= 1, 'The number of threads has to be > 1'

 with cf.ProcessPoolExecutor(max_workers=args.n) as pool:
  results = pool.map(fib, [args.number] * args.n)
"""
text_file = open("multiprocess.py", "w")
n = text_file.write(s)
text_file.close()

In [ ]:
!time python multiprocess.py -n 2 34
!time python multiprocess.py -n 4 34


real	0m6.464s
user	0m6.423s
sys	0m0.036s

real	0m6.700s
user	0m12.835s
sys	0m0.042s


In [ ]:

s ="""
import concurrent.futures as cf

def fib(n):
  if n<=2:
    return 1
  if n==0:
    return 0
  return fib(n-1)+fib(n-2)


if __name__ == '__main__':
 import argparse
 parser = argparse.ArgumentParser()
 parser.add_argument('-n', type=int, default=1)
 parser.add_argument('number', type=int, nargs='?', default=34)
 args = parser.parse_args()
 assert args.n >= 1, 'The number of threads has to be > 1'

 with cf.ThreadPoolExecutor(max_workers=args.n) as pool:
  results = pool.map(fib, [args.number] * args.n)
"""'''  '''
text_file = open("multiprocess_t.py", "w")
n = text_file.write(s)
text_file.close()

In [ ]:
!time python multiprocess_t.py -n 2 34
!time python multiprocess_t.py -n 4 34


real	0m4.395s
user	0m4.263s
sys	0m0.031s

real	0m8.071s
user	0m7.837s
sys	0m0.056s


In [ ]:
#multiprocess ques

In [ ]:

s ="""
import multiprocessing as mp

def fib(n):
  if n<=2:
    return 1
  if n==0:
    return 0
  return fib(n-1)+fib(n-2)

def worker(inq,outq):
  while True:
    data = inq.get()
    if data is None:
      return
    fn,arg=data
    outq.put(fn(arg))

if __name__ == '__main__':
 import argparse
 parser = argparse.ArgumentParser()
 parser.add_argument('-n', type=int, default=1)
 parser.add_argument('number', type=int, nargs='?', default=34)
 args = parser.parse_args()
 assert args.n >= 1, 'The number of threads has to be > 1'

 task = mp.Queue()
 results = mp.Queue()

 for i in range(args.n):
   mp.Process(target=worker,args=(task,results)).start()

 for i in range(args.n):
   print(results.get())

 for i in range(args.n):
   task.put(None)

"""
text_file = open("multiprocess_q.py", "w")
n = text_file.write(s)
text_file.close()

In [ ]:
!time python multiprocess_q.py -n 2 4